### 模型函数

In [1]:
'bab'.split('\n')

['bab']

In [9]:
from tts import TTS
mytts = TTS()
# model_path = {'gpt_path':'E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\训练集_denoise\yueer_denoise-e25.ckpt', 'sovits_path':'E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\训练集_denoise\yueer_denoise_e25_s575.pth'}
# mytts.modify_model(model_path)
# mytts.run('你好，你现在可以跟我进行对话了')
text = '''卡方分箱的差异值就是卡方值，它表示每一对相邻分箱中的观察频数与期望频数之间的偏离程度 。卡
方值越小，说明两个分箱之间的差异越小，可以合并为一个分箱；卡方值越大，说明两个分箱之间的
差异越大，应该保持分开 。
卡方分箱需要引入目标值，因为它是基于卡方检验的独立性检验来实现的，即要判断某个数值变量是
否会影响目标变量。比如，在评分卡建模中，目标变量通常是是否违约，数值变量可以是年龄、收入'''
splits = {'\n', ',','.','。','，','!','！', '?', '？',':', '：'}
def fun():
    count =''
    for i in text:
        count +=i
        if i in splits:
            mytts.run(count[:-1])
            count =''
        yield i
for i in fun():
    print(i, end='')

卡方分箱的差异值就是卡方值，它表示每一对相邻分箱中的观察频数与期望频数之间的偏离程度 。卡
方值越小，说明两个分箱之间的差异越小，可以合并为一个分箱；卡方值越大，说明两个分箱之间的
差异越大，应该保持分开 。
卡方分箱需要引入目标值，因为它是基于卡方检验的独立性检验来实现的，即要判断某个数值变量是
否会影响目标变量。比如，在评分卡建模中，目标变量通常是是否违约，数值变量可以是年龄、收入

In [ ]:
from model import Model
import re,sys
messages =[]
cache_path = "E:\LargeModel\Gradio_Python\weibui\cache"
mymodel = Model(cache_path,messages)
mymodel.disply_inf=False
bot = mymodel.bot
history=[[]]
while(True):
    question=input()
    if len(question)==0:
        break
    history +=[[question, None]]
    print('user:', question)
    print('Bot：', end='')
    for word in bot(history):
        print('\r', flush=True, end='')
        print('Bot：'+word[-1][1],end='')
        print('\r', end='',flush=True)
        print('\r', end='',flush=True)
        # sys.stdout.flush()
    print()
    history = word

### 更改参数

In [ ]:
import gradio as gr
import json
def gr_cut_params(model, max_epoch, cut_nepoch):
    #生成cut_params
    if model=="ollama":
        #cut_nepoch随着max_epoch改变
        max_epoch = gr.Slider(minimum=1,maximum=100,step=1,label='max_epoch',value=15,interactive=True, visible=True)
        cut_nepoch = gr.Slider(minimum=0,maximum=15,step=1,label='cut_nepoch',value=5,interactive=True, visible=True)
    elif model =="None":
        max_epoch = gr.Slider(visible=False)
        cut_nepoch = gr.Slider(visible=False)
    return(max_epoch, cut_nepoch)

def cut_epoch(max_epoch, cut_nepoch):
    #根据max_epoch生成cut_nepoch
    cut_nepoch = gr.Slider(minimum=0,maximum=max_epoch,step=1,label='cut_nepoch',value=max(max_epoch//3,1),interactive=True, visible=True)
    return(cut_nepoch)

def save_cut_params(max_epoch, cut_nepoch):
    cut_params = {
        'max_epoch':max_epoch,
        'cut_nepoch':cut_nepoch
    }
    with open('.\cache\cut_params.json','w') as file:
        file.write(json.dumps(cut_params))
    return('params has been saved')


def gr_model_params(model,
                num_predict,
                temperature,
                top_p,
                top_k,
                num_ctx,
                repeat_penalty,
                seed,
                num_gpu,
                stop
                ):
    if model =='None':
        num_predict = gr.Slider(visible=False)
        temperature = gr.Slider(visible=False)
        top_p = gr.Slider(visible=False)
        top_k = gr.Slider(visible=False)
        num_ctx = gr.Slider(visible=False)
        repeat_penalty = gr.Slider(visible=False)
        seed = gr.Number(visible=False)
        num_gpu = gr.Number(visible=False)
        stop = gr.Textbox(visible=False)
    elif model =="ollama":
        ###num_predict, num_ctx都是2的幂次, num_gpu要判断是否为-1，stop要按\n拆分--model.py---done
        num_predict = gr.Slider(minimum=5,maximum=14,step=1,label='2^num_predict',value=7,interactive=True, visible=True)
        temperature = gr.Slider(minimum=0.1,maximum=1,step=0.1,label='temperature',value=0.8,interactive=True, visible=True)
        top_p = gr.Slider(minimum=0.1,maximum=1,step=0.1,label='top_p',value=0.9,interactive=True, visible=True)
        top_k = gr.Slider(minimum=5,maximum=100,step=5,label='top_k',value= 40,interactive=True, visible=True)
        num_ctx = gr.Slider(minimum=5,maximum=14,step=1,label='num_ctx',value=11,interactive=True, visible=True)
        repeat_penalty = gr.Slider(minimum=0.3,maximum=2,step=0.1,label='repeat_penalty',value=1.1,interactive=True,visible=True)
        seed = gr.Number(label='seed',value=0,visible=True)
        num_gpu = gr.Number(label='num_gpu',value=-1,visible=True)
        stop = gr.Textbox(label="shift+enter to input more stop text. default:AI assistant:", value="AI assistant:", lines=2, visible=True)

    return(num_predict,
            temperature,
            top_p,
            top_k,
            num_ctx,
            repeat_penalty,
            seed,
            num_gpu,
            stop
            )

def save_params(
                num_predict,
                temperature,
                top_p,
                top_k,
                num_ctx,
                repeat_penalty,
                seed,
                num_gpu,
                stop
                ):
    params = {'num_predict':num_predict, 
                'temperature':temperature,
                'top_p':top_p,
                'top_k':top_k,
                'num_ctx':num_ctx,
                'repeat_penalty':repeat_penalty,
                'seed':seed,
                'num_gpu':num_gpu,
                'stop':stop
            }
    with open('.\cache\params.json','w') as file:
        file.write(json.dumps(params))
    return('params has been saved')
    
import gradio as gr
import ollama
def found_model_loader():
    #待办：model_loader待传入json
    model_loader = ['None', 'ollama']
    model_loader = gr.Dropdown(model_loader, label='model_loader')
    return(model_loader)

def found_model():
    model = gr.Dropdown(['None'], label='model')
    return(model)

def model_select_update(model_loader, model):
    # model_loader与model的级联界面，借鉴自https://zhuanlan.zhihu.com/p/663411336
    if model_loader=='None':
        models_list = ['None']
    elif model_loader =='ollama':
        #待办：可以事先写入一个json文件
        models_list = ['None']+[model['name'] for model in  ollama.list()['models']]
    model = gr.Dropdown.update(choices=models_list)
    return(model)

def save_model(model):
    with open(r'.\cache\model_val.txt', 'a', encoding='utf-8') as file:
        file.write(model + "\n") 
    return

def save_model_loader(model_loader):
    with open(r'.\cache\model_loader_val.txt', 'a', encoding='utf-8') as file:
        file.write(model_loader + "\n") 
    return    

In [ ]:
import gradio as gr
from model import Model
from load_model import found_model, found_model_loader, model_select_update, save_model,save_model_loader,save_promot_text
from model_loader_params import gr_cut_params,gr_model_params,cut_epoch,save_params,save_cut_params

with gr.Blocks() as h:
   promot_text = gr.Textbox(label='promot_text', value="", lines=1)
   promot_text.submit(save_promot_text, inputs=[promot_text], outputs=None)

   #选择框架
   model_loader = found_model_loader()
   #选择模型
   model = found_model()
   model_loader.change(model_select_update, inputs=[model_loader, model], outputs=[model])
   #存储
   model_loader.change(save_model_loader, inputs=[model_loader], outputs=None)
   model.change(save_model, inputs=[model], outputs=None)

   # #选择最大对话轮次，超过时裁剪对话记录
   max_epoch, cut_nepoch = gr.Slider(visible=False),gr.Slider(visible=False)
   model_loader.change(gr_cut_params, inputs=[model_loader, max_epoch, cut_nepoch], outputs=[max_epoch, cut_nepoch])
   max_epoch.change(cut_epoch, inputs=[max_epoch, cut_nepoch], outputs=[cut_nepoch])
   gr.Interface(save_cut_params, inputs=[max_epoch, cut_nepoch], outputs='text')

   #选择模型参数---待办：参数用params代替
   num_predict = gr.Slider(visible=False)
   temperature = gr.Slider(visible=False)
   top_p = gr.Slider(visible=False)
   top_k = gr.Slider(visible=False)
   num_ctx = gr.Slider(visible=False)
   repeat_penalty = gr.Slider(visible=False)
   seed = gr.Number(visible=False)
   num_gpu = gr.Number(visible=False)
   stop = gr.Textbox(visible=False)
   model_loader.change(gr_model_params, inputs=[model_loader, num_predict,temperature,top_p,top_k,num_ctx, repeat_penalty, seed,num_gpu,stop], outputs=[num_predict,temperature,top_p,top_k,num_ctx, repeat_penalty, seed,num_gpu,stop])
   #将参数值存储cache/params.json
   gr.Interface(save_params, inputs=[num_predict,temperature,top_p,top_k,num_ctx, repeat_penalty, seed,num_gpu,stop], outputs='text')
         # 读取参数
h.launch()